In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
from time import time
from copy import deepcopy
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT_PATH = Path().absolute().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
from frequency_response import FrequencyResponse
from peq import PEQ
from constants import DEFAULT_FS, PEQ_CONFIGS

In [4]:
dir_path = Path().absolute()
config_names = [
    ['4_PEAKING_WITH_LOW_SHELF'],
    ['4_PEAKING_WITH_LOW_SHELF', '4_PEAKING_WITH_HIGH_SHELF'],
    ['8_PEAKING_WITH_SHELVES'],
    ['10_BAND_GRAPHIC_EQ'],
]

In [5]:
def rmse(peq):
    ix10k = np.argmin(np.abs(10000 - peq.f))
    return np.sqrt(np.mean(np.square(peq.target[:ix10k] - peq.fr[:ix10k])))

In [7]:
def band_reqularization_stats(configs, max_n=None):
    df = []
    n = 0
    for fp in ROOT_PATH.joinpath('results', 'oratory1990').glob('*/*/*.csv'):
        fr = FrequencyResponse.read_from_csv(fp)
        t = time()
        peqs = fr.optimize_parametric_eq(configs, DEFAULT_FS, max_time=5)
        df.append([time() - t, rmse(peqs[-1])])
        n += 1
        if max_n is not None and n >= max_n:
            break
    df = pd.DataFrame(df, columns=['Time', 'RMSE'])
    return df.describe()

## No Band Reqularization
Comment out band penalties from filters in `PEQ._optimizer_loss()`

In [20]:
band_reqularization_stats([PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.109325,0.471122
std,0.045137,0.182414
min,0.048011,0.235244
25%,0.074270,0.362158
50%,0.103523,0.449119
75%,0.143782,0.541614
max,0.182041,0.895833


## Band Reqularization
Do **NOT** comment out band penalties from filters in `PEQ._optimizer_loss()`

In [21]:
band_reqularization_stats([PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.140021,0.645004
std,0.037645,0.216923
min,0.075017,0.308490
25%,0.134280,0.443026
50%,0.145533,0.722022
75%,0.161786,0.822237
max,0.197572,0.902935


In [22]:
band_reqularization_stats([PEQ_CONFIGS['4_PEAKING_WITH_LOW_SHELF']], max_n=10)

,Time,RMSE
count,10.000000,10.000000
mean,0.503678,0.645004
std,0.138125,0.216923
min,0.265059,0.308490
25%,0.472515,0.443026
50%,0.524389,0.722022
75%,0.582521,0.822237
max,0.695156,0.902935
